### ETL to get the text data from the playlist

This notebook shows the process of building the corpus of transcripts from the YouTube playlist.

**Extract**: Pull data (transcripts) from each video.  
**Transform**:  
**Load**: Load data into our database where it will be retrieved from.  

In [1]:
from etl import youtube
import json

First we load the video information. This includes the video IDs and titles.

In [2]:
with open('data/example_videos.json') as f:
    video_info = json.load(f)

Then we must extract the transcripts using the YouTube Transcript API. This is done over all of the videos.  
This produces a list of video segments with timestamps.  
Next, we format the transcript by adding metadata so that the segments are easily identified for retreival later.  
Finally, embeddings of the transcripts are created using the conv-bert-base model from HuggingFace.

In [3]:
videos = []
for video in video_info:
    video_id = video["id"]
    video_title = video["title"]
    transcript = youtube.get_video_transcript(video_id)
    if transcript:
        formatted_transcript = youtube.format_transcript(transcript, video_id, video_title)
        transcript_with_embeddings = youtube.add_embeddings_to_data(formatted_transcript)
        videos.extend(transcript_with_embeddings)

The last step is to load the data into a database. We will use a SQLite database.

In [4]:
import sqlite3

# Clear the database 
conn = sqlite3.connect('data/etl_example.db')
cursor = conn.cursor()

cursor.execute('''
    DELETE FROM videos
''')

conn.commit()
conn.close()

# Load the data into the database
youtube.load_data_to_db('data/etl_example.db', videos)

Data loaded to database at data/etl_example.db.


Here is some of the data:

In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/etl_example.db')
cursor = conn.cursor()

cursor.execute('''
    SELECT text, embedding, metadata
    FROM videos
''')

transcript_data = cursor.fetchall()

cursor.close()

transcript_df = pd.DataFrame(transcript_data, columns=['text', 'embedding', 'metadata'])

transcript_df.head()

,text,embedding,metadata
0,"Hello, and welcome. As you probably know, deep...",b'D5\x96>\xb4\x1f7>\xae6\x82=\xac\x0b\x13<\xd4...,"{""video_id"": ""CS4cs9xVecg"", ""segment_id"": 1, ""..."
1,But deep learning is also enabling brand new p...,b'\xdf*\xca>\xbfJE=\xa6\x13\xf8=\xb8\xf4F\xbb\...,"{""video_id"": ""CS4cs9xVecg"", ""segment_id"": 2, ""..."
2,Everything ranging from better healthcare wher...,b'\xf7;_\xbdW\xd0\xac\xbc\xfd\x16\xfb\xbd\x8f\...,"{""video_id"": ""CS4cs9xVecg"", ""segment_id"": 3, ""..."
3,"delivering personalized education, to precisio...",b'\x900\xe5=\xbd\x9e\xf3=@\xd5\xe3\xbd\xca\x88...,"{""video_id"": ""CS4cs9xVecg"", ""segment_id"": 4, ""..."
4,to even self-driving cars and many others.,b'x%\x81>l\xfe;=\xf0\xdd\x9d<{\x96\xaf\xbb\xd0...,"{""video_id"": ""CS4cs9xVecg"", ""segment_id"": 5, ""..."
